In [89]:
# !pip install friends
import pandas as pd

train_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_train.csv')
val_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_val.csv')
test_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_test.csv')

with open('./data/wikilarge/wiki_train.src', 'r', encoding='utf-8') as f:
    train_src = pd.DataFrame(f.readlines(), columns=['src'])
with open('./data/wikilarge/wiki_val.src', 'r', encoding='utf-8') as f:
    val_src = pd.DataFrame(f.readlines(), columns=['src'])
with open('./data/wikilarge/wiki_test.src', 'r', encoding='utf-8') as f:
    test_src = pd.DataFrame(f.readlines(), columns=['src'])

with open('./data/wikilarge/wiki_train.tgt', 'r', encoding='utf-8') as f:
    train_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])
with open('./data/wikilarge/wiki_val.tgt', 'r', encoding='utf-8') as f:
    val_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])
with open('./data/wikilarge/wiki_test.tgt', 'r', encoding='utf-8') as f:
    test_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])

train_data = pd.concat([train_info, train_src, train_tgt], axis=1)
val_data = pd.concat([val_info, val_src, val_tgt], axis=1)
test_data = pd.concat([test_info, test_src, test_tgt], axis=1)

all_data = pd.concat([train_data, val_data, test_data], axis=0)
# subset to src tgt and grade
all_data = all_data[['src', 'tgt', 'abs_tgt_FKGL_Grade', 'abs_src_FKGL_Grade']]
# drop duplicates
all_data = all_data[all_data['src'] != all_data['tgt']]

In [ ]:
from sklearn.model_selection import train_test_split
from collections import defaultdict
from datasets import DatasetDict, Dataset
import pickle
groups = all_data.groupby('abs_tgt_FKGL_Grade')

group_dict = defaultdict()
split_dict = defaultdict()
for group in groups:
    group_dict[group[0]] = group[1].reset_index(drop=True)

for group in group_dict.keys():
    train, test_val = train_test_split(group_dict[group], test_size=1000, random_state=42)
    val, test = train_test_split(test_val, test_size=0.5, random_state=42)
    split_dict[group] = {'train': train.reset_index(), 'val': val.reset_index(), 'test': test.reset_index()}

# convert dataframes to datasets and save to pickle format
for group in split_dict.keys():
    print(group, len(split_dict[group]['train']), len(split_dict[group]['val']), len(split_dict[group]['test']))
    train_data = Dataset.from_pandas(split_dict[group]['train'])
    val_data = Dataset.from_pandas(split_dict[group]['val'])
    test_data = Dataset.from_pandas(split_dict[group]['test'])
    dataset_dict = DatasetDict({'train': train_data, 'val': val_data, 'test': test_data})
    with open(f'./data/wikilarge/graded_splits/grade_{group}.pkl', 'wb') as f:
        pickle.dump(dataset_dict, f)


0 10062 500 500
1 4425 500 500
2 7363 500 500
3 9668 500 500
4 19995 500 500
5 12856 500 500
6 24563 500 500
7 17589 500 500
8 22712 500 500
9 15058 500 500
10 17535 500 500
11 8169 500 500
12 9968 500 500
13 24288 500 500
